# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [ ]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [ ]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [ ]:
y.value_counts()

1    71
0    59
2    48
Name: target, dtype: int64

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.25)

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [ ]:
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,12.972857,2.386842,2.362180,19.433835,100.759398,2.277068,2.021203,0.363534,1.608647,5.017594,0.959444,2.610000,742.992481
std,0.829993,1.098905,0.280606,3.467312,14.999571,0.645696,1.005537,0.126923,0.576964,2.202516,0.234545,0.729961,306.867593
min,11.030000,0.890000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.420000,1.740000,0.480000,1.270000,278.000000
25%,12.290000,1.640000,2.210000,17.200000,88.000000,1.700000,1.200000,0.260000,1.250000,3.250000,0.780000,1.830000,500.000000
50%,12.990000,1.900000,2.360000,19.400000,98.000000,2.230000,2.140000,0.340000,1.560000,4.800000,0.970000,2.810000,675.000000
75%,13.690000,3.170000,2.540000,21.500000,108.000000,2.800000,2.880000,0.450000,1.960000,6.130000,1.120000,3.200000,970.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,10.800000,1.710000,4.000000,1547.000000


In [ ]:
X_train.head(10)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
100,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0
122,12.42,4.43,2.73,26.5,102.0,2.20,2.13,0.43,1.71,2.08,0.92,3.12,365.0
154,12.58,1.29,2.10,20.0,103.0,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640.0
51,13.83,1.65,2.60,17.2,94.0,2.45,2.99,0.22,2.29,5.60,1.24,3.37,1265.0
76,13.03,0.90,1.71,16.0,86.0,1.95,2.03,0.24,1.46,4.60,1.19,2.48,392.0
56,14.22,1.70,2.30,16.3,118.0,3.20,3.00,0.26,2.03,6.38,0.94,3.31,970.0
26,13.39,1.77,2.62,16.1,93.0,2.85,2.94,0.34,1.45,4.80,0.92,3.22,1195.0
153,13.23,3.30,2.28,18.5,98.0,1.80,0.83,0.61,1.87,10.52,0.56,1.51,675.0
138,13.49,3.59,2.19,19.5,88.0,1.62,0.48,0.58,0.88,5.70,0.81,1.82,580.0


In [ ]:
X_train.proline.mean(), X_train.magnesium.mean()

(742.9924812030075, 100.7593984962406)

## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

In [ ]:
X_train.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.226328,-0.024519,1.101127,-0.241394,0.016101,0.812935,1.216668,-0.502463,2.090070,0.301887,0.301960,0.770065,1.445831
1,-1.079807,-0.280281,-2.368742,-0.559842,-0.251581,-0.073170,0.148537,-0.818807,-0.362996,-0.782781,1.329088,0.481291,-0.107920
2,-0.668617,1.866297,1.315759,2.045642,0.083022,-0.119807,0.108607,0.525654,0.176331,-1.338787,-0.168807,0.701309,-1.236434
3,-0.475115,-1.001897,-0.937868,0.163904,0.149942,-1.239098,-1.438686,1.316514,-0.362996,1.176913,-1.623905,-1.457623,-0.336894
4,1.036615,-0.673059,0.850725,-0.646691,-0.452343,0.268836,0.967105,-1.135151,1.185393,0.265427,1.200697,1.045088,1.707515


In [ ]:
X_test.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,0.806832,0.651425,0.707637,-1.225688,1.019911,0.657478,1.007035,-1.530581,0.089343,0.037556,0.002381,1.031337,0.333672
1,1.496181,1.510056,0.278375,-0.154544,0.685308,0.890664,0.627699,-0.502463,-0.623960,0.101360,-0.382792,0.990084,1.102370
2,-0.051831,0.386528,1.208443,0.453402,-0.318502,-1.145824,-1.518546,1.316514,-1.493842,-0.190315,-0.810762,-0.412535,-0.467736
3,0.915677,-0.810075,1.208443,0.887649,0.016101,1.123849,1.226651,-0.581549,1.341972,0.311002,0.986712,0.137512,1.772936
4,-0.729086,-1.111509,-1.581761,0.048105,-1.523073,-0.259718,-0.021166,-0.739721,-0.989310,-0.153856,0.687133,1.196351,-0.762131


Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier().fit(X_train, y_train)

In [ ]:
from os import XATTR_CREATE
y_pred = model.predict(X_test)

In [ ]:
y_pred

array([0, 0, 2, 0, 1, 0, 1, 2, 1, 2, 0, 2, 0, 2, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 2, 2, 2, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 1, 1, 1,
       2])

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [ ]:
from sklearn.metrics import f1_score

old_score = f1_score(y_test, y_pred, average='weighted')
old_score

0.9550512333965844

## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neigbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [ ]:
n_neighbors = np.arange(3,30,2)

In [ ]:
from tables.file import parameters
parameters = {'n_neighbors': n_neigbors, 'weights':('uniform', 'distance')}

In [ ]:
from sklearn.model_selection import GridSearchCV


new_model = KNeighborsClassifier()
gs = GridSearchCV(new_model, parameters, cv=3, scoring='f1_weighted')

In [ ]:
new_model.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [ ]:
gs.fit(X_train, y_train)

gs.best_score_, gs.best_params_

(0.9623290498688744, {'n_neighbors': 19, 'weights': 'distance'})

Заново обучите KNN с наилучшими по `GridSearch` гиперпараметрами на тренировочных данных,  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [ ]:
best_model = KNeighborsClassifier(weights='distance', n_neighbors=19).fit(X_train, y_train)

In [ ]:
best_pred = best_model.predict(X_test)

In [ ]:
best_score = f1_score(y_test, best_pred, average='weighted')
best_score

0.9550512333965844

**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

In [ ]:
old_score == best_score

True

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, best_pred)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]])

In [ ]:
y_test.value_counts()

1    18
0    15
2    12
Name: target, dtype: int64

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, best_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      0.89      0.94        18
           2       0.92      1.00      0.96        12

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [ ]:
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями
    
    return data[inds_k], dists[inds_k]

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [ ]:
q = X_test.iloc[0].values

In [ ]:
q

array([ 0.80683207,  0.65142516,  0.70763713, -1.22568751,  1.01991065,
        0.65747817,  1.00703506, -1.53058075,  0.08934271,  0.03755595,
        0.00238119,  1.03133729,  0.33367219])

In [ ]:
my_data = X.iloc[0].values

In [ ]:
%%time

neighbors, dists = knn_search(q, my_data)  
for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

top 1: dist = 291.3390837689502
top 2: dist = 303.3201417434126
top 3: dist = 325.5425282871958
top 4: dist = 328.0761764817218
top 5: dist = 337.08695864809624
CPU times: user 1.13 ms, sys: 0 ns, total: 1.13 ms
Wall time: 975 µs


Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [ ]:
q

array([ 0.80683207,  0.65142516,  0.70763713, -1.22568751,  1.01991065,
        0.65747817,  1.00703506, -1.53058075,  0.08934271,  0.03755595,
        0.00238119,  1.03133729,  0.33367219])

In [ ]:
neighbors[0]

array([ 12.  ,   0.92,   2.  ,  19.  ,  86.  ,   2.42,   2.26,   0.3 ,
         1.43,   2.5 ,   1.38,   3.12, 278.  ])

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [ ]:
def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

In [ ]:
def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0
    
    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals
    
    return hash_key

In [ ]:
def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)
        
    return hash_table

In [ ]:
def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

In [ ]:
%%time

neighbors, dists = approx_knn_search(q, my_data)

for i, (neighbor, dist) in enumerate(zip(neighbors, dists)):
    print(f"top {i + 1}: dist = {dist}")

IndexError: ignored

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?